In [201]:
import re
import os
import sys
# import string as str
import numpy as np
import pandas as pd
from sklearn import *
# from tensorflow import *
from sklearn.metrics.pairwise import cosine_similarity as cossim
from sklearn.feature_extraction.text import CountVectorizer as cvector
from sklearn.model_selection import train_test_split as tts

In [202]:
df=pd.read_csv("bestsellers with categories.csv").dropna().drop_duplicates()
print(df.shape)
df.head()

(550, 7)


,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


In [203]:
train, test = tts(df,test_size=300,shuffle=True)
test["book_id"]=range(0,test.shape[0])
test=test.set_index("book_id")
print(test.shape)
test.head()

(300, 7)


,Name,Author,User Rating,Reviews,Price,Year,Genre
book_id,,,,,,,
0,The Mueller Report,The Washington Post,4.6,2744,12,2019,Non Fiction
1,The Going-To-Bed Book,Sandra Boynton,4.8,5249,5,2017,Fiction
2,Magnolia Table: A Collection of Recipes for Ga...,Joanna Gaines,4.8,9867,16,2018,Non Fiction
3,Dog Man: Brawl of the Wild: From the Creator o...,Dav Pilkey,4.9,7235,4,2019,Fiction
4,Astrophysics for People in a Hurry,Neil deGrasse Tyson,4.7,9374,9,2017,Non Fiction


In [ ]:
columns=["Name","Price","Genre"]
test[columns].head()
df.info()
test.info()

In [ ]:
def get_important_features(data):
    important_features = []
    for index,row in data.iterrows():
        new_column = re.sub('[^A-Za-z0-9]+' , " " , row["Name"])\
        +" "+re.sub('[^A-Za-z0-9]+' , " " , row["Genre"])\
        +" "+(row["Price"]).__str__()\
        # +" "+(row['User Rating'])
        important_features.append(new_column.lower())
    return important_features

test["important_features"]=get_important_features(test)
test.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre,important_features
book_id,,,,,,,,
0,Percy Jackson and the Olympians Paperback Boxe...,Rick Riordan,4.8,548,2,2010,Fiction,percy jackson and the olympians paperback boxe...
1,Love You Forever,Robert Munsch,4.8,18613,5,2014,Fiction,love you forever fiction 5
2,"Act Like a Lady, Think Like a Man: What Men Re...",Steve Harvey,4.6,5013,17,2009,Non Fiction,act like a lady think like a man what men real...
3,Daring Greatly: How the Courage to Be Vulnerab...,Brené Brown,4.8,1329,10,2013,Non Fiction,daring greatly how the courage to be vulnerabl...
4,"Oh, the Places You'll Go!",Dr. Seuss,4.9,21834,8,2012,Fiction,oh the places you ll go fiction 8


In [ ]:
cm=cvector().fit_transform(test["important_features"])
cs=cossim(cm)
print(cs.shape)
cs

(100, 100)


array([[1.        , 0.15811388, 0.13187609, ..., 0.1118034 , 0.3       ,
        0.28284271],
       [0.15811388, 1.        , 0.20851441, ..., 0.1767767 , 0.15811388,
        0.2236068 ],
       [0.13187609, 0.20851441, 1.        , ..., 0.07372098, 0.13187609,
        0.09325048],
       ...,
       [0.1118034 , 0.1767767 , 0.07372098, ..., 1.        , 0.1118034 ,
        0.15811388],
       [0.3       , 0.15811388, 0.13187609, ..., 0.1118034 , 1.        ,
        0.28284271],
       [0.28284271, 0.2236068 , 0.09325048, ..., 0.15811388, 0.28284271,
        1.        ]])

# recommendation start

In [ ]:
PersonA_want="Night (Night)"
PersonB_want="The Twilight Saga Collection"

book_prefer_id_for_A=test[test.Name==PersonA_want].index.values[0]
book_prefer_id_for_B=test[test.Name==PersonB_want].index.values[0]


In [ ]:
book_prefer_id_for_A

30

In [ ]:
book_prefer_id_for_B

59

In [ ]:
scores=list(enumerate(cs[book_prefer_id_for_A]))
scores

[(0, 0.12909944487358058),
 (1, 0.20412414523193154),
 (2, 0.17025130615174974),
 (3, 0.17407765595569785),
 (4, 0.1543033499620919),
 (5, 0.12909944487358058),
 (6, 0.2721655269759087),
 (7, 0.11322770341445959),
 (8, 0.10910894511799621),
 (9, 0.2886751345948129),
 (10, 0.20412414523193154),
 (11, 0.2461829819586655),
 (12, 0.1543033499620919),
 (13, 0.08512565307587487),
 (14, 0.1873171623163388),
 (15, 0.3333333333333334),
 (16, 0.2721655269759087),
 (17, 0.20412414523193154),
 (18, 0.2357022603955159),
 (19, 0.2886751345948129),
 (20, 0.1666666666666667),
 (21, 0.2357022603955159),
 (22, 0.1873171623163388),
 (23, 0.17817416127494962),
 (24, 0.2461829819586655),
 (25, 0.18257418583505539),
 (26, 0.21081851067789195),
 (27, 0.2357022603955159),
 (28, 0.12909944487358058),
 (29, 0.25819888974716115),
 (30, 1.0000000000000002),
 (31, 0.21821789023599242),
 (32, 0.16329931618554525),
 (33, 0.22645540682891918),
 (34, 0.11785113019775795),
 (35, 0.18257418583505539),
 (36, 0.3086066999

In [ ]:
sorted_scores = sorted(scores, key=lambda x:x[1],reverse=True)
sorted_scores

[(30, 1.0000000000000002),
 (41, 0.4082482904638631),
 (43, 0.36514837167011077),
 (15, 0.3333333333333334),
 (94, 0.3333333333333334),
 (36, 0.3086066999241838),
 (65, 0.3086066999241838),
 (89, 0.3086066999241838),
 (9, 0.2886751345948129),
 (19, 0.2886751345948129),
 (46, 0.2886751345948129),
 (72, 0.2886751345948129),
 (74, 0.2886751345948129),
 (76, 0.2886751345948129),
 (6, 0.2721655269759087),
 (16, 0.2721655269759087),
 (45, 0.2721655269759087),
 (29, 0.25819888974716115),
 (11, 0.2461829819586655),
 (24, 0.2461829819586655),
 (69, 0.2461829819586655),
 (75, 0.2461829819586655),
 (18, 0.2357022603955159),
 (21, 0.2357022603955159),
 (27, 0.2357022603955159),
 (47, 0.2357022603955159),
 (55, 0.2357022603955159),
 (58, 0.2357022603955159),
 (61, 0.2357022603955159),
 (66, 0.2357022603955159),
 (80, 0.2357022603955159),
 (86, 0.2357022603955159),
 (33, 0.22645540682891918),
 (44, 0.22645540682891918),
 (64, 0.22645540682891918),
 (85, 0.22645540682891918),
 (88, 0.2264554068289191

In [ ]:
i=0
print("Top 10 recommendations for the books (",PersonA_want, ") are:\n")
for each_book in sorted_scores:
    
    PersonB_want=test[test.index==each_book[0]]["Author"].values[0]
    print(i,PersonB_want)
    i=i+1
    if i>10:
        break

Top 10 recommendations for the books ( Night (Night) ) are:

0 Elie Wiesel
1 Gallup
2 Roger Priddy
3 Stephen Kendrick
4 Bruce Springsteen
5 Justin Halpern
6 W. Cleon Skousen
7 DK
8 Rob Elliott
9 The College Board
10 Malcolm Gladwell


In [ ]:
def sort_scores(book_prefer_id):
    scores=list(enumerate(cs[book_prefer_id]))

    sorted_scores=sorted(scores,key=lambda x:x[1],reverse=True)
    return sorted_scores[1:]

def print_top_ten(sorted_scores,book_prefer_id,j=1):
    print("Top 10 recommendation to movie (", book_prefer_id,") are:\n")
    for each_book in sorted_scores:
        book_prefer_id=test[test.index==each_book[0]]['Name'].values[0]
        print(j,book_prefer_id)
        j+=1
        if j>10:break

def see_top_ten_books_for(some_book):
    print("===============================")
    print("looking for the book = "+some_book)
    book_prefer_id=None
    try:
        book_prefer_id=test[test.Name==some_book].index.values[0]
    except:
        print("Book not Found")
    if book_prefer_id is not None:
        print("Fount it")
        scores=sort_scores(book_prefer_id)
        print_top_ten(scores,some_book)
    print("===============================")

In [ ]:
see_top_ten_books_for("Gone Girl")

looking for the book = Gone Girl
Fount it
Top 10 recommendation to movie ( Gone Girl ) are:

1 Wonder
2 Old School (Diary of a Wimpy Kid #10)
3 The Girl Who Played with Fire (Millennium Series)
4 The Guernsey Literary and Potato Peel Pie Society
5 Proof of Heaven: A Neurosurgeon's Journey into the Afterlife
6 The Girl with the Dragon Tattoo (Millennium Series)
7 Divergent
8 The Getaway
9 Goodnight Moon
10 The Help


In [ ]:
see_top_ten_books_for("Giraffes Can't Dance")

looking for the book = Giraffes Can't Dance
Book not Found


In [ ]:
dm = pd.read_csv("bestsellers with categories copy.xlsx")

In [ ]:
dm

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction
...,...,...,...,...,...,...,...
545,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction
546,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2016,Non Fiction
547,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2017,Non Fiction
548,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2018,Non Fiction
